In [1]:
import spacy
from spacy import displacy

from preprocessing import (
    WordCaser,
    PolishLemmatizer,
    PolishLetterReplacer,
    PunctuationRemover,
    StopWordsRemover,
    CurrencyRemover,
    UnitsRemover,
    NumberReplacer,
    WhitespaceRemover
)

import warnings
warnings.filterwarnings("ignore")

nlp = spacy.load('pl_core_news_lg')

In [2]:
import pandas as pd
dataset=pd.read_csv('polish_dataset.csv')
dataset.head()

,description,length,rate
0,Polecam nie pierwszy i nie ostatni raz!,39.0,1.0
1,Bardzo dobra komunikacja sms i telefoniczna. Z...,121.0,1.0
2,Polecam zakupy w tym sklepie. Są dostępne częś...,87.0,1.0
3,0,0.0,0.0
4,Jestem w pełni zadowolona z przebiegu transakcji,48.0,1.0


In [3]:
del dataset['length']

dataset.dropna(inplace=True)
dataset = dataset[dataset.rate != 0]

del dataset['rate']

In [4]:
dataset.head()

,description
0,Polecam nie pierwszy i nie ostatni raz!
1,Bardzo dobra komunikacja sms i telefoniczna. Z...
2,Polecam zakupy w tym sklepie. Są dostępne częś...
4,Jestem w pełni zadowolona z przebiegu transakcji
5,Transakcja przebiegła błyskawicznie. Pełen pro...


In [5]:
dataset.shape

(917641, 1)

In [ ]:
from sklearn.pipeline import make_pipeline
additional_punctuation = ['\\.', '\\+', '/', '%']


preprocessing_pipe = make_pipeline(
    WordCaser(),
    PolishLemmatizer(),
    PolishLetterReplacer(),
    PunctuationRemover(additional_punctuation=additional_punctuation),
    StopWordsRemover(),
    CurrencyRemover(),
    UnitsRemover(),
    NumberReplacer(),
    WhitespaceRemover()
)

description_transformed = preprocessing_pipe.fit_transform(dataset['description'].values)

In [7]:
description_transformed

['polecac pierwszy ostatni',
 'dobry komunikacja sms telefoniczny zamowiony towar wyslany termin zabezpieczony polecac sklep',
 'polecac zakup sklep dostepny czesc stary model np lodowka',
 'pelnia zadowolony przebieg transakcja',
 'transakcja przebiec blyskawicznie pelny profesjonalizm polecac sklep',
 'obsluga godny polecenie klient tranzakcja przebiec szybko bezproblemowo',
 'obsluga czas realizacja poziom zapakowania paczka wzorcowo polecac',
 'sprawnie bezproblemowo',
 'transakcja przebiegac b sprawnie produkt przesylka b szybko polecac',
 'pierwszy odebrac przesylka paczkomat wrocic sprzedawca sklep wyslac ponownie przesylka dodatkowy oplata p l e c',
 'dobry kontakt sklep szybki wysylka nastepny dzien wplata miec produkt zamawiac szyba piekarnik dokladnie zabezpieczona polecac',
 'sprawnie milo polecac zdarzyc blad reklamacja zalatwiac reka problem',
 'transakcja przebiec pomyslnie',
 'rzetelny konkretny elastyczny obsluga wyrozumiec decyzja kupujacego polecac potrzeba wracac za

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

In [23]:
vectorizer = TfidfVectorizer(max_features=1500, min_df=10)
X = vectorizer.fit_transform(description_transformed)
words = np.array(vectorizer.get_feature_names())

In [24]:
words

array(['absolutnie', 'adres', 'agd', ..., 'zyd', 'zydowski', 'zyto'],
      dtype='<U16')

In [25]:
nmf = NMF(n_components=50, solver="mu")
W = nmf.fit_transform(X)
H = nmf.components_

for i, topic in enumerate(H):
     print("Topic {}: {}".format(i + 1, ",".join([str(x) for x in words[topic.argsort()[-10:]]])))

Topic 1: zostalo,towar,informacja,dobry,zgodny,obsluga,zrealizowane,szybki,realizacja,zamowienie
Topic 2: rzetelnie,zalatwione,przeprowadzona,milo,pojsc,zrealizowane,przebiec,polecac,szybko,sprawnie
Topic 3: wspolpraca,ksiegarnia,porzadek,komunikacja,porzadku,internetowy,sklep,obsluga,cena,dobry
Topic 4: kompetentny,rewelacyjny,rzetelny,towar,polecac,profesjonalny,fachowy,sklep,mily,obsluga
Topic 5: zamowiony,zabezpieczony,zdjecie,dostarczony,dobry,wysylka,zapakowany,opis,towar,zgodny
Topic 6: sprzedawca,wzorowo,blyskawicznie,pozytywnie,tranzakcja,prawidlowo,polecac,transakcja,pomyslnie,przebiec
Topic 7: solidny,przyjemnosc,korzystac,pierwszy,towar,fajny,kupowac,zakup,internetowy,sklep
Topic 8: rewelacyjny,korzystny,wysylka,szybki,przystepny,polecac,atrakcyjny,konkurencyjny,niski,cena
Topic 9: rewelacja,blyskawicznie,czysty,sumienie,zakup,kazdemu,szczerze,zdecydowanie,serdecznie,polecac
Topic 10: xd,zamowic,czekac,dostac,ciag,godzina,zadowolenie,rok,dzien,number
Topic 11: zamawiac,uslu